In [1]:
import pandas as pd

Primeiro vamos analisar quais são os possiveis dados a serem analisados

### PRIMEIRA IDEIA

In [2]:
# O Dicionário de dados pede para que o encoding seja esse

dataFrame = pd.read_csv('ProuniRelatorioDadosAbertos2020.csv', sep=';', encoding='windows-1252') 

dataFrame.head(5)

,ANO_CONCESSAO_BOLSA,CODIGO_EMEC_IES_BOLSA,NOME_IES_BOLSA,MUNICIPIO,CAMPUS,TIPO_BOLSA,MODALIDADE_ENSINO_BOLSA,NOME_CURSO_BOLSA,NOME_TURNO_CURSO_BOLSA,CPF_BENEFICIARIO,SEXO_BENEFICIARIO,RACA_BENEFICIARIO,DATA_NASCIMENTO,BENEFICIARIO_DEFICIENTE_FISICO,REGIAO_BENEFICIARIO,UF_BENEFICIARIO,MUNICIPIO_BENEFICIARIO
0,2020,322,UNIVERSIDADE PAULISTA,IPATINGA,IPATINGA,INTEGRAL,EAD,PEDAGOGIA,CURSO A DISTÂNCIA,991.XXX.XXX-91,F,Parda,11/08/1973,N,SUDESTE,MG,GOVERNADOR VALADARES
1,2020,163,UNIVERSIDADE ESTÁCIO DE SÁ,FORTALEZA,EAD VIA CORPVS - CE,INTEGRAL,EAD,MARKETING,CURSO A DISTÂNCIA,067.XXX.XXX-01,M,Parda,13/05/1987,N,NORDESTE,CE,FORTALEZA
2,2020,17670,FACULDADE DE QUIXERAMOBIM,QUIXERAMOBIM,FACULDADE DE QUIXERAMOBIM - UNIQ,INTEGRAL,PRESENCIAL,FARMÁCIA,NOTURNO,623.XXX.XXX-27,M,Parda,23/07/2001,N,NORDESTE,CE,MOMBACA
3,2020,203,UNIVERSIDADE SÃO JUDAS TADEU,SAO PAULO,PAULISTA,PARCIAL,PRESENCIAL,DIREITO,MATUTINO,089.XXX.XXX-40,F,Branca,04/04/2003,N,NORDESTE,BA,IBITITA
4,2020,203,UNIVERSIDADE SÃO JUDAS TADEU,SAO PAULO,PAULISTA,INTEGRAL,PRESENCIAL,DIREITO,MATUTINO,173.XXX.XXX-09,F,Branca,07/12/1977,N,SUDESTE,SP,SAO PAULO


É possível observar que existem diversas possibilidades de análise, uma vez que o gráfico possui sexo, região, curso, raça e data de nascimento.

Como foi pedida uma análise com data, uma função de data, temos apenas um campo para lidar com isso.

In [3]:
dataFrame['DATA_NASCIMENTO'].dtype

dtype('O')

Como o campo não é do tipo 'date', vou precisar transformar. Para isso, acredito ter que manipular como string para trocar o mês com o dia e a função converter de forma correta.

Depois disso, bastará apenas categorizar. Menos de 20 anos, menos de 30 anos, menos de 40 anos e mais de 40 anos serão as categorias.

Nessa ideia teremos:
 - uma função de string para trocar o mês com o dia
 - uma função para converter para data
 - uma função de condicional para criar as classificações

### SEGUNDA IDEIA

É de conhecimento de todos que o curso de medicina é bastante concorrido, pois, para o brasileiro médio, é praticamente impossível o pagamento de uma faculdade de medicina. Por ser concorrido, a tendência é que haja uma eletização do curso, já que os mais abastados tem acesso a melhor educação. Dessa forma, seriam as vagas de medicina ocupadas por pessoas predominantemente brancas, mesmo com as cotas?

In [4]:
# Pegando as colunas de curso e raça pra que faz medicina
dataFrame[dataFrame['NOME_CURSO_BOLSA'] == 'MEDICINA'][['NOME_CURSO_BOLSA','RACA_BENEFICIARIO']]

,NOME_CURSO_BOLSA,RACA_BENEFICIARIO
1399,MEDICINA,Parda
1400,MEDICINA,Branca
1401,MEDICINA,Branca
1402,MEDICINA,Branca
1403,MEDICINA,Branca
...,...,...
160986,MEDICINA,Branca
160987,MEDICINA,Branca
160988,MEDICINA,Branca
166672,MEDICINA,Parda


Nessa ideia, será possível utilizar dois ou mais operadores lógicos, pois nessa pergutam existem, no mínimo, duas condições. 

### TERCEIRA IDEIA

Como os dados de região das pessoas são disponibilizados:

In [5]:
# Pegando as colunas de curso e região
dataFrame[['REGIAO_BENEFICIARIO','NOME_CURSO_BOLSA']]

,REGIAO_BENEFICIARIO,NOME_CURSO_BOLSA
0,SUDESTE,PEDAGOGIA
1,NORDESTE,MARKETING
2,NORDESTE,FARMÁCIA
3,NORDESTE,DIREITO
4,SUDESTE,DIREITO
...,...,...
166825,SUDESTE,ADMINISTRAÇÃO
166826,SUDESTE,RELAÇÕES INTERNACIONAIS
166827,SUL,ADMINISTRAÇÃO
166828,NORTE,RELAÇÕES INTERNACIONAIS


É possível fazer um rank com os top 3 cursos mais escolhidos por região, fazendo um agrupamento com group by.

### Tratar o data frame

##### Verificar dados irrelevantes

In [6]:
dataFrame["ANO_CONCESSAO_BOLSA"].value_counts() # Todos são de 2020, o que torna essa coluna irrelevante

ANO_CONCESSAO_BOLSA
2020    166830
Name: count, dtype: int64

A solução que darei quando irei fazer a análise será remover essa coluna

##### Descobrir colunas com valores nulos

In [7]:
dataFrame.describe(include='all') # Descrição estatística de todas as colunas

,ANO_CONCESSAO_BOLSA,CODIGO_EMEC_IES_BOLSA,NOME_IES_BOLSA,MUNICIPIO,CAMPUS,TIPO_BOLSA,MODALIDADE_ENSINO_BOLSA,NOME_CURSO_BOLSA,NOME_TURNO_CURSO_BOLSA,CPF_BENEFICIARIO,SEXO_BENEFICIARIO,RACA_BENEFICIARIO,DATA_NASCIMENTO,BENEFICIARIO_DEFICIENTE_FISICO,REGIAO_BENEFICIARIO,UF_BENEFICIARIO,MUNICIPIO_BENEFICIARIO
count,166830.0,166830.000000,166830,166830,166830,166830,166830,166792,166830,166830,166830,166830,166830,166830,166830,166830,166830
unique,NaN,NaN,1376,1434,6462,2,2,317,5,41390,2,6,11793,2,5,27,4885
top,NaN,NaN,UNIVERSIDADE PAULISTA,SAO PAULO,UNIDADE SEDE,INTEGRAL,PRESENCIAL,DIREITO,NOTURNO,076.XXX.XXX-00,F,Parda,31/12/1969,N,SUDESTE,SP,SAO PAULO
freq,NaN,NaN,16043,18663,17841,130744,113005,15082,75329,37,101137,78067,362,165558,71644,40379,11469
mean,2020.0,2270.745022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,4063.329275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2020.0,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2020.0,322.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2020.0,1196.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2020.0,1952.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Nome do curso é a unica coluna que tem menos valores

dataFrame['NOME_CURSO_BOLSA'].isnull().value_counts() # Nome do curso tem valores nulos

NOME_CURSO_BOLSA
False    166792
True         38
Name: count, dtype: int64

A solução que encontrei será substituir esses dados nulos pelo que valor que mais aparece na coluna

### Colocando arquivo no bucket

In [9]:
from dotenv import load_dotenv
import os
import boto3

load_dotenv(override=True)  # Isso carrega as variáveis do arquivo .env


# Access keys hardcoded (não seguro)
s3 = boto3.client('s3',
   aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
   aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    aws_session_token=os.environ.get("AWS_SECRET_SESSION_TOKEN"),
   region_name=os.environ.get("AWS_REGION")
)


bucket_name = "meu-bucket-unico-gabriel-2025"  # nome globalmente único
region = os.environ.get("AWS_REGION")

try:
    s3.upload_file("ProuniRelatorioDadosAbertos2020.csv", "meu-bucket-dados-prouni", "dadosProuni.csv")
    print(f"✅ Bucket '{bucket_name}' criado com sucesso!")
except Exception as e:
    print(f"❌ Erro ao criar bucket: {e}")

✅ Bucket 'meu-bucket-unico-gabriel-2025' criado com sucesso!
